In [22]:
### How to log 
### How to register a data
### How to retrieve data from datastore / workspace (registered)
### Check all experiments in a workspace
### Check all runs in a experiment
### Get a specific run by its tag/property
### Tag runs, add properties

### How to register a model
### Review all registered models
### Retrieve a model by its tag


### How to create a new environment
### How to register a enviroment


### How to create a new compute cluster
### How to create a run config for pipelines


In [1]:
from azureml.core import Workspace, Experiment, Run, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails
import os
import sys

In [22]:
script_dir = 'playground'
os.makedirs(script_dir, exist_ok=True)


In [33]:
%%writefile $script_dir/data_prep.py
import os
import argparse
from azureml.core import Run
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

parser = argparse.ArgumentParser()
parser.add_argument('--input-data', type=str, dest='raw_data', help='Raw dataset')
parser.add_argument('--preped-data', type=str, dest='preped_data', help='Prepared dataset')
args = parser.parse_args()

saved_dir = args.preped_data
run = Run.get_context()

df = run.input_datasets['raw_data'].to_pandas_dataframe()

num_features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure',
                'TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
scaler = MinMaxScaler()
df[num_features] = scaler.fit(df[num_features])

print('saving data to preped_data')
os.makedirs(saved_dir, exist_ok=True)
df.to_csv(os.path.join(saved_dir, 'preped_data.csv'), index=False)
run.complete()

Overwriting playground/data_prep.py


In [39]:
%%writefile $script_dir/train.py

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import argparse
import numpy as np
from azureml.core import Run, Model
import joblib

parser = argparse.ArgumentParser()
parser.add_argument('--training-data', type=str, dest='input_data', help='Input dataset for model')
args = parser.parse_args()

training_data_dir = args.input_data

run = Run.get_context()

# !!!!!
# df = run.input_datasets['input_data'].to_pandas_dataframe()
df = pd.read_csv(os.path.join(training_data_dir, 'preped_data.csv'))

X = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin',
            'BMI','DiabetesPedigree','Age']].values

y = df['Diabetic'].values


X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=.3, random_state=0)

model = DecisionTreeClassifier().fit(X_train, y_train)

y_pred = model.predict(X_test)

# ACC
acc = np.average(y_pred==y_test)
run.log(name='acc', value=acc)

# AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
run.log(name='auc', value=auc)

# ROC
fpr, tpr, threshold = roc_curve(y_test, y_score[:,1])
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot([0,1], [0,1], 'k--')
ax.plot(fpr, tpr)
ax.title('ROC curve')
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
run.log_image(name='roc_curve', plot=fig)

print('Saving model')
os.makedirs('models', exist_ok=True)

model_file_path = os.path.join('models', 'diabetes_model.pkl')
joblib.dump(model, model_file_path)

Model.register(workspace=ws,
              model_path=model_file,
              tags={"training context": "pipeline"},
              properties={'AUC': auc, "ACC": acc},
              model_Path=model_file_path)
run.complete()


Overwriting playground/train.py


In [5]:
ws = Workspace.from_config()
datastore = ws.get_default_datastore()


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [6]:
for data in ws.datasets:
    print(data)


diabetes_dataset2
batch-data
diabetes file dataset
diabetes dataset
loan_data


In [7]:
'diabetes dataset' in ws.datasets

True

In [8]:
from glob import glob
# [os.path.join(root, f) for f in files for root, dirs, files in os.walk('data')]
glob('data/*.csv')

['data\\diabetes.csv', 'data\\diabetes2.csv']

In [9]:
# register dataset
from azureml.core import Dataset
from azureml.data.datapath import DataPath

dataset_name = 'diabetes_dataset2'
# if this dataset already registered, we will do nothing, otherwise, we upload the files, and register it.
# if the file already uploaded, we will be notified, without re-uploading it again.
if dataset_name not in ws.datasets: 
    print("We will upload the files to cloud datastore, retrive it and register it")
    Dataset.File.upload_directory(src_dir='data', target=DataPath(datastore, dataset_name)) 
    tab_data = Dataset.Tabular.from_delimited_files(path=(datastore, os.path.join(dataset_name, '*.csv')))
    try:
        tab_data = tab_data.register(workspace=ws,
                                    name=dataset_name,
                                    tags={"format": "csv"},
                                    description="diabetes dataset",
                                    create_new_version=True)
    except Exception as ex:
        print(ex)
else:
    print("dataset already exist")
    


dataset already exist


In [10]:
# get a registered dataset from workspace
ws.datasets.get(dataset_name).to_pandas_dataframe().head(10)

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0
5,1619297,0,82,92,9,253,19.724160,0.103424,26,0
6,1660149,0,133,47,19,227,21.941357,0.174160,21,0
7,1458769,0,67,87,43,36,18.277723,0.236165,26,0
8,1201647,8,80,95,33,24,26.624929,0.443947,53,1
9,1403912,1,72,31,40,42,36.889576,0.103944,26,0


### Prepare the environment

In [11]:
%%writefile $script_dir/experiment_env.yml
name: experiment_env_2
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Overwriting playground/experiment_env.yml


In [12]:
from azureml.core import Environment
# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env_2", script_dir + "/experiment_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env_2')

### Prepare a compute instance

In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ComputerCluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


### Setup the run config for the pipeline

In [14]:
from azureml.core.runconfig import RunConfiguration
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


#### Define all the PythonScriptStep and intermediate data reference

In [40]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
preped_data = OutputFileDatasetConfig("preped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = script_dir,
                                script_name = "data_prep.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('raw_data'),
                                             '--preped-data', preped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = script_dir,
                                script_name = "train.py",
                                arguments = ['--training-data', preped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [41]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

Pipeline is built.


In [42]:
# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-diabetes-pipeline-2')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Created step Prepare Data [ec052f9b][afe7fb5c-0987-4c55-8d3d-84cd577a714a], (This step will run and generate new outputs)
Created step Train and Register Model [5d742752][05d07e3b-f6a4-4e8d-ae8f-1a9afe388bf7], (This step will run and generate new outputs)
Submitted PipelineRun b235f721-9fef-4dfb-8291-513abc8a77bf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b235f721-9fef-4dfb-8291-513abc8a77bf?wsid=/subscriptions/efaef50b-3a01-4bf1-ad06-b63c101ab300/resourcegroups/resource-group-1/workspaces/myworkspace&tid=cb956b3e-0e1a-485c-a395-a000041d2695
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: b235f721-9fef-4dfb-8291-513abc8a77bf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b235f721-9fef-4dfb-8291-513abc8a77bf?wsid=/subscriptions/efaef50b-3a01-4bf1-ad06-b63c101ab300/resourcegroups/resource-group-1/workspaces/myworkspace&tid=cb956b3e-0e1a-485c-a395-a000041d2695
PipelineRun Status: Running


StepRunId: aca64d49-6a40-4b98-8efe-20d77cbf5965
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/aca64d49-6a40-4b98-8efe-20d77cbf5965?wsid=/subscriptions/efaef50b-3a01-4bf1-ad06-b63c101ab300/resourcegroups/resource-group-1/workspaces/myworkspace&tid=cb956b3e-0e1a-485c-a395-a000041d2695
StepRun( Prepare Data ) Status: NotStarted
StepRun( Prepare Data ) Status: Running

StepRun(Prepare Data) Execution Summary
StepRun( Prepare Data ) Status: Finished
{'runId': 'aca64d49-6a40-4b98-8efe-20d77cbf5965', 'target': 'ComputerCluster', 'status': 'Completed', 'startTimeUtc': '2023-01-15T18:14:07.814031Z', 'endTimeUtc': '2023-01-15T18:15:13.720986Z', '



PipelineRun Execution Summary
PipelineRun Status: Failed


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "messageFormat": "Pipeline has some failed steps. {0}",
        "messageParameters": {},
        "referenceCode": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus2",
    "location": "eastus2",
    "time": "2023-01-15T18:15:36.815921Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Pipeline has some failed steps. See child run or execution logs for more details.\",\n        \"messageFormat\": \"Pipeline has some failed steps. {0}\",\n        \"messageParameters\": {},\n        \"referenceCode\": \"PipelineHasStepJobFailed\",\n        \"details\": []\n    },\n    \"environmen

In [ ]:
tab_data.to_pandas_dataframe().head(10)